In [16]:
import requests
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import re

In [17]:
DATA_PATH = './data/'
DATA = DATA_PATH + 'BindingDB_All.tsv'

## Let's look for a suitable protein family to study

Step 1. Match targets to their families. We made a choice to use PANTHER classification, although other classifications exist too.

In [18]:
df = pd.read_csv(DATA, sep="\t", on_bad_lines='skip', low_memory=False, dtype={
    "BindingDB Reactant_set_id": np.int32,
})


In [19]:
df

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.12,UniProt (SwissProt) Entry Name of Target Chain.12,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12
0,1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,InChI=1S/C31H42N2O7/c34-27(35)17-9-3-11-19-32-...,XGEGDSLAQZJGCW-HHGOQMMWSA-N,608734,"6-[(4R,5S,6S,7R)-4,7-dibenzyl-3-(5-carboxypent...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,UZLMEAPBHYEHAC-UNTBESQGSA-N,22,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,InChI=1S/C29H34N4O3/c34-27-25(16-21-8-3-1-4-9-...,HYNYUFZPPJMPOB-UTWJFGBXSA-N,23,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,InChI=1S/C29H40N2O4/c32-18-10-2-1-9-17-30-25(1...,YXVAZXDWVZTGGD-VIJSPRBVSA-N,24,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,InChI=1S/C28H38N2O4/c31-17-9-3-8-16-29-24(18-2...,WWTSWTPNILRSJX-XDZXDJIYSA-N,25,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927359,51531022,CC[C@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc(...,InChI=1S/C24H26F3N3O2S/c1-3-19-14-29(13-17-9-1...,ZBKXSRSXIGZPCY-IBGZPJMESA-N,50615961,CHEMBL5268424,Peroxisome proliferator-activated receptor gamma,Mus musculus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927360,51531023,CC[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc...,InChI=1S/C24H26F3N3O2S/c1-3-19-14-29(13-17-9-1...,ZBKXSRSXIGZPCY-LJQANCHMSA-N,50615962,CHEMBL5272996,Peroxisome proliferator-activated receptor gamma,Mus musculus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927361,51531024,CC(C)[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2...,InChI=1S/C25H28F3N3O2S/c1-15(2)21-14-30(13-18-...,RVZUBLMVVIWYLZ-NRFANRHFSA-N,50615964,CHEMBL5277165,Peroxisome proliferator-activated receptor gamma,Mus musculus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927362,51531025,COc1ccc(cc1)N(C)c1nc(C)nc2[nH]ccc12,InChI=1S/C15H16N4O/c1-10-17-14-13(8-9-16-14)15...,HKPVMULFKBBABY-UHFFFAOYSA-N,50615967,CHEMBL1275943,Similar to alpha-tubulin isoform 1,NaN,NaN,2600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
import sys
sys.path.append('./src/utils')
from retrieve_family import retrieve_family

uniprot_ids = df["UniProt (SwissProt) Entry Name of Target Chain"].unique()
# Retrieve family information for the test ID
family_dict = await retrieve_family(uniprot_ids)


100%|██████████| 6533/6533 [00:04<00:00, 1556.44it/s]


In [43]:
#map each list to a string

# List of all unique protein families
families = set()
for key in family_dict:
    families.update(family_dict[key])

# Create a dictionary with the protein families as keys and the proteins as values
family_dict_2 = {family: [] for family in families}
for key in family_dict:
    for family in family_dict[key]:
        family_dict_2[family].append(key)
            
for key in family_dict:
    family_dict[key] = '||'.join(family_dict[key])

with open("protein_families.json", "w") as f:
    json.dump(family_dict, f, indent=2)
    print("Protein families saved to 'protein_families.json'")


Protein families saved to 'protein_families.json'


In [45]:
family_dict_2

{'TRANSCRIPTION FACTOR SPT20 HOMOLOG': ['KAT2B_HUMAN',
  'GCN5_YEAST',
  'KAT2A_HUMAN'],
 'DNA POLYMERASE III SUBUNIT BETA': ['DPO3_BACSU',
  'DPO3B_ECOLI',
  'DPO3_STAAU'],
 'NUCLEOPORIN NUP188 HOMOLOG': ['NEP_HUMAN'],
 'GLUTAMATE RECEPTOR-INTERACTING PROTEIN GRIP': ['EPHB2_MOUSE',
  'KIF5A_HUMAN',
  'GRIP1_RAT',
  'GRIP1_RAT',
  'GRIA2_HUMAN',
  'GRIA2_HUMAN',
  'PRLHR_HUMAN',
  'PRLHR_HUMAN',
  'PLCD4_RAT',
  'EPHA7_HUMAN',
  'GRIA2_RAT',
  'GRIA2_RAT',
  'GRIA4_RAT',
  'EPHB2_HUMAN',
  'GRIA3_RAT',
  'GRIA3_RAT',
  'GRIA3_HUMAN',
  'GRIA3_HUMAN',
  'PTPRF_HUMAN',
  'EPHA7_MOUSE',
  'GRM7_RAT',
  'PTPRF_RAT',
  'XDH_HUMAN',
  'VDR_MOUSE',
  'KIF5C_HUMAN'],
 'SOLUTE CARRIER FAMILY 35': ['BECN1_HUMAN',
  'FFAR2_HUMAN',
  'TTPA_HUMAN',
  'DRD1_MOUSE',
  'CNR2_HUMAN',
  'ACSL5_HUMAN',
  'ASGR1_HUMAN',
  'DRD1_HUMAN',
  'C1S_HUMAN'],
 'ADIPOCYTE-SPECIFIC PROTEIN 4-RELATED': ['SARM1_HUMAN', 'IRAK1_HUMAN'],
 'ZONA PELLUCIDA SPERM-BINDING PROTEIN 3': ['VDAC2_HUMAN'],
 'SPERMATOGENESIS-ASSOC

In [46]:
df_family = pd.DataFrame.from_dict(family_dict, orient='index', columns=['Family'])
df_family


,Family
HRH4_MOUSE,5-HYDROXYTRYPTAMINE RECEPTOR||5-HYDROXYTRYPTAM...
5HT7R_RAT,5-HYDROXYTRYPTAMINE RECEPTOR
PD2R2_HUMAN,G-PROTEIN COUPLED RECEPTOR FAMILY 1 MEMBER||CH...
DPO3_BACSU,DNA POLYMERASE III SUBUNIT ALPHA||DNA POLYMERA...
PYRF_METTH,OROTIDINE 5'-PHOSPHATE DECARBOXYLASE
...,...
CCDC6_HUMAN,H4 D10S170 PROTEIN-RELATED||SMALL NUCLEAR RNA ...
FNTA_YEAST,PROTEIN FARNESYLTRANSFERASE ALPHA SUBUNIT/RAB ...
1433S_HUMAN,14-3-3 PROTEIN
MMP12_RAT,MATRIX METALLOPROTEINASE||METALLOPROTEASE INHI...


In [47]:
df_family = pd.DataFrame.from_dict(family_dict, orient='index', columns=['Family'])
# Number of unique protein families
print(f"There are {len(families)} unique families in this dataset")
print(f"Number of NaNs in 'Family' column: {df_family['Family'].isna().sum()}")

# Map for each family the number of proteins in the dataset (Family column is a || separated string)
family_counts = {}
for family in families:
    if family not in family_counts:
        family_counts[family] = 0
    family_counts[family] += int(df_family['Family'].str.contains(re.escape(family)).sum())



There are 5307 unique families in this dataset
Number of NaNs in 'Family' column: 0


In [48]:
print(f"Number of proteins in each family in descending order:")
family_counts = dict(sorted(family_counts.items(), key=lambda item: item[1], reverse=True))
for family in family_counts:
    print(f"{family}: {family_counts[family]}")

Number of proteins in each family in descending order:
-: 4521
CLE: 994
UBIQUITIN: 542
G-PROTEIN COUPLED RECEPTOR: 461
ACTIN: 431
UNCHARACTERIZED: 378
SERINE/THREONINE-PROTEIN KINASE: 348
TYROSINE-PROTEIN KINASE: 345
XIN: 286
PROTEIN PHOSPHATASE: 280
DEHYDROGENASE: 222
ZINC FINGER PROTEIN: 210
METHYLTRANSFERASE: 208
TYROSINE-PROTEIN KINASE RECEPTOR: 207
LIPASE: 190
RIBOSOMAL PROTEIN S6 KINASE: 185
UBIQUITIN CARBOXYL-TERMINAL HYDROLASE: 170
MITOGEN-ACTIVATED PROTEIN KINASE: 162
NUCLEAR HORMONE RECEPTOR: 161
HOMEOBOX PROTEIN: 147
METALLOPROTEASE: 145
ADRENERGIC RECEPTOR-RELATED G-PROTEIN COUPLED RECEPTOR: 131
EUKARYOTIC TRANSLATION INITIATION FACTOR: 128
TRANSMEMBRANE PROTEIN: 120
CELL ADHESION MOLECULE: 116
CELL DIVISION PROTEIN KINASE: 109
MAP/MICROTUBULE AFFINITY-REGULATING KINASE: 104
E3 UBIQUITIN-PROTEIN LIGASE TRIM: 101
TYROSINE-PROTEIN PHOSPHATASE: 101
CYTOKINE RECEPTOR: 95
NEUROTRANSMITTER GATED ION CHANNEL: 94
IMMUNOGLOBULIN: 94
ARRESTIN: 94
SERINE PROTEASE: 92
CYCLIN: 89
5-HYDR

In [49]:
# Only keep families with at least n targets
n = 5
# drop if family is ""
df_family = df_family[df_family['Family'] != ""]
df_family_filtered = df_family[df_family['Family'].apply(lambda x: any([family_counts[family] >= n for family in x.split('||')]))]
print(f"Number of families with at least {n} targets: {len(df_family_filtered)}")

Number of families with at least 5 targets: 5638


In [50]:
# In the original dataframe, only keep the rows with UniProt IDs that have a family
df_filtered = df[df['UniProt (SwissProt) Entry Name of Target Chain'].isin(df_family.index)]
# Add the family column to the original dataframe
df_filtered = df_filtered.join(df_family, on='UniProt (SwissProt) Entry Name of Target Chain')

In [51]:
# Check the amount of missing Ki
print(f"Number of NaNs in 'Ki (nM)' column: {df_filtered['Ki (nM)'].isna().sum()}")
# Check the amount of missing IC50
print(f"Number of NaNs in 'IC50 (nM)' column: {df_filtered['IC50 (nM)'].isna().sum()}")
# Drop the rows with missing IC50
df_filtered = df_filtered.dropna(subset=['IC50 (nM)'])

Number of NaNs in 'Ki (nM)' column: 2313133
Number of NaNs in 'IC50 (nM)' column: 960211


In [54]:
# family_dict = family -> [protein1, protein2, protein3]
# df_grouped = family -> (each column of df_filtered counts the number of proteins in the family)
# df filtered only has a || separated list of families

dict_grouped = {}
family_dict_3 = family_dict_2.copy()
for family in family_dict_3:
    family_dict_3[family] = set(family_dict_3[family])

# compute ratio
for family in families:
    dict_grouped[family] = {"IC50 (nM)": 0, "UniProt IDs": 0, 'Ligand to Target Ratio' : 0}
    dict_grouped[family]["IC50 (nM)"] = len(family_dict_2[family])
    dict_grouped[family]["UniProt IDs"] = len(family_dict_3[family])
    dict_grouped[family]['Ligand to Target Ratio'] = len(family_dict_2[family])/len(family_dict_3[family])




In [55]:
df_grouped = pd.DataFrame.from_dict(dict_grouped, orient='index')
df_grouped = df_grouped.sort_values('Ligand to Target Ratio', ascending=False)
df_grouped

,IC50 (nM),UniProt IDs,Ligand to Target Ratio
QUINOLENE RESISTANCE PROTEIN NORA,24,1,24.0
CYCLIC NUCLEOTIDE-GATED ION CHANNEL 15-RELATED-RELATED,22,1,22.0
G-PROTEIN COUPLED RECEPTOR MTH-RELATED,21,1,21.0
RNA POLYMERASE-BINDING TRANSCRIPTION FACTOR DKSA-RELATED,20,1,20.0
PRESEQUENCE PROTEASE,40,2,20.0
...,...,...,...
PHYTOENE DESATURASE,2,2,1.0
TRANSMEMBRANE PROTEIN WITH METALLOPHOSPHOESTERASE DOMAIN-RELATED,2,2,1.0
CALBINDIN,1,1,1.0
FETAL AND ADULT TESTIS-EXPRESSED TRANSCRIPT PROTEIN,5,5,1.0


In [ ]:
# Save the filtered dataframe to a new file
df_grouped.to_csv(DATA_PATH + 'BindingDB_counts.tsv', sep='\t', index=False)

In [ ]:
# Show proteins belonging to PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN
df_filtered[df_filtered['Family'] == 'PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN']['UniProt (SwissProt) Entry Name of Target Chain'].unique()

array(['BTK_HUMAN', 'PLCG1_BOVIN', 'PLCB1_HUMAN', 'PLCB_HUMAN'],
      dtype=object)

In [ ]:
# Find "PLC" in UniProt IDs
winners = df_filtered[df_filtered['UniProt (SwissProt) Entry Name of Target Chain'].str.contains('PLC', na=False)]['UniProt (SwissProt) Entry Name of Target Chain'].unique()

In [ ]:
# Compute the number of occurencies of each winner
test = ['BTK_HUMAN']
df_filtered[df_filtered['UniProt (SwissProt) Entry Name of Target Chain'].isin(test)].groupby('UniProt (SwissProt) Entry Name of Target Chain').count()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Entry Name of Target Chain.12,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12,Family
UniProt (SwissProt) Entry Name of Target Chain,,,,,,,,,,,,,,,,,,,,,
BTK_HUMAN,22095,22095,16849,16849,22095,22095,22095,19969,0,22095,...,0,0,0,0,0,0,0,0,0,22095


In [ ]:
response = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query=BTK_HUMAN&fields=xref_panther",
                        headers={"Accept": "application/json"})
if response.status_code == 200:
    results = response.json().get("results", [])
results

[{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
  'primaryAccession': 'Q06187',
  'uniProtKBCrossReferences': [{'database': 'PANTHER',
    'id': 'PTHR24418',
    'properties': [{'key': 'EntryName', 'value': 'TYROSINE-PROTEIN KINASE'},
     {'key': 'MatchStatus', 'value': '1'}]},
   {'database': 'PANTHER',
    'id': 'PTHR24418:SF92',
    'properties': [{'key': 'EntryName',
      'value': 'TYROSINE-PROTEIN KINASE BTK'},
     {'key': 'MatchStatus', 'value': '1'}]}],
  'extraAttributes': {'uniParcId': 'UPI0000126B16'}},
 {'entryType': 'UniProtKB reviewed (Swiss-Prot)',
  'primaryAccession': 'P78347',
  'uniProtKBCrossReferences': [{'database': 'PANTHER',
    'id': 'PTHR46304:SF2',
    'properties': [{'key': 'EntryName',
      'value': 'GENERAL TRANSCRIPTION FACTOR II-I'},
     {'key': 'MatchStatus', 'value': '1'}]},
   {'database': 'PANTHER',
    'id': 'PTHR46304',
    'properties': [{'key': 'EntryName',
      'value': 'GENERAL TRANSCRIPTION FACTOR II-I REPEAT DOMAIN-CONTAINING PROTEIN 1